In [7]:
import kfp
import os

In [8]:
# Load DKube components
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")

In [9]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token)

In [12]:
@kfp.dsl.pipeline(
    name='heart-1',
    description='heart disease pipeline with dkube components'
)
def heart_pipeline():
    train = dkube_training_op(auth_token = token,container = '{"image":"ocdr/d3-datascience-sklearn:v0.23.2-1"}',
                                    framework="sklearn", version="0.23.2",
                                    program="heart-disease-prediction", run_script="python train.py",
                                    datasets='["heart-disease-dataset"]', outputs='["heart-disease-prediction-model"]',
                                    input_dataset_mounts='["/dataset"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]')

    serving = dkube_serving_op(auth_token = token,model = train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/sklearnserver:0.23.2"}',
                                    transformer_image='{"image":"ocdr/d3-datascience-sklearn:v0.23.2-1"}',
                                    transformer_project="heart-disease-prediction",
                                    transformer_code='transform.py').after(train)

In [13]:
client.create_run_from_pipeline_func(heart_pipeline, arguments={})

RunPipelineResult(run_id=94326274-8c34-4123-b287-e6acc5887928)